<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2020SightVisit_kmnist_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資格スクエア G 検定対策ビデオ教材 の kmninst プロトタイプ

- filename: `2020SightVisit_kmnist_prototype.ipynb`
- author: 浅川伸一

---



In [ ]:
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz
!wget http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz

In [ ]:
import numpy as np

def load(f):
    return np.load(f)['arr_0']

# Load the data
Xkm_train = load('kmnist-train-imgs.npz')
Xkm_test = load('kmnist-test-imgs.npz')
ykm_train = load('kmnist-train-labels.npz')
ykm_test = load('kmnist-test-labels.npz')

# Flatten images
n_samples = 2000
#x_train = x_train.reshape(-1, 784)[:n_samples]
#y_train = y_train[:n_samples]
#x_test = x_test.reshape(-1, 784)

ind2c =[c for c in 'おきすつなまはやれを']

In [ ]:
!pip install japanize-matplotlib
import japanize_matplotlib

In [ ]:
# ライブラリの輸入
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision 
from torch.utils.data import DataLoader, Dataset
from PIL import Image
#from torch.optim.lr_scheduler import StepLR

# imports
import torch
import torchvision.transforms as transforms
#import glob
import matplotlib.pyplot as plt
import numpy as np
#import torchvision
#import time
#import albumentations as A
#import tarfile


In [ ]:
# define pytorch transforms
# source: https://debuggercafe.com/image-augmentation-using-pytorch-and-albumentations/
# and https://albumentations.ai/docs/getting_started/image_augmentation/
transform = transforms.Compose([
     transforms.ToPILImage(),
     transforms.Resize((300, 300)),
     transforms.CenterCrop((100, 100)),
     transforms.RandomCrop((80, 80)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(degrees=(-90, 90)),
     transforms.RandomVerticalFlip(p=0.5),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

In [ ]:
# PyTorch image augmentation module
class PyTorchImageDataset(Dataset):
    def __init__(self, image_list, transforms=None):
        self.image_list = image_list
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.image_list))
    
    def __getitem__(self, i):
        image = self.image_list[i]
        #image = plt.imread(self.image_list[i])
        image = Image.fromarray(image).convert('RGB')        
        image = np.asarray(image).astype(np.uint8)
        if self.transforms is not None:
            image = self.transforms(image)
            
        return torch.tensor(image, dtype=torch.float)

In [ ]:
pytorch_dataset = PyTorchImageDataset(image_list=Xkm_train, transforms=None)
pytorch_dataloader = DataLoader(dataset=pytorch_dataset, batch_size=16, shuffle=True)

In [ ]:
def show_img(img):
    #plt.figure(figsize=(18,15))
    # unnormalize
    img = img / 2 + 0.5  
    npimg = img.numpy()
    npimg = np.clip(npimg, 0., 1.)
    #plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.imshow(np.transpose(npimg, (0, 1, 2)))
    plt.show()

In [ ]:
data = iter(pytorch_dataloader)
images = data.next()

# show images
#plt.imshow(np.asarray(images[0].numpy().astype(np.uint8)))
show_img(images[0:3])

In [ ]:
#Xkm_train.shape
plt.imshow(Xkm_train[0])

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)  # input channels, output channels, kernel size, stride
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


In [ ]:
import time

start = time.time()
for i, data in enumerate(pytorch_dataloader):
    images = data
end = time.time()
time_spent = (end-start)/60
print(f"{time_spent:.3} minutes")